<a href="https://colab.research.google.com/github/MuhammadAbuSufian/colab/blob/main/AGIQA_3K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import pandas as pd
from transformers import AutoProcessor, AutoModel
from PIL import Image
import torch
import requests
from io import BytesIO
import os

# Setup device for model computation
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model and processor
processor_name_or_path = "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"
model_pretrained_name_or_path = "yuvalkirstain/PickScore_v1"
processor = AutoProcessor.from_pretrained(processor_name_or_path)
model = AutoModel.from_pretrained(model_pretrained_name_or_path).eval().to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

In [2]:

# Function to calculate probabilities
def calc_probs(prompt, images):
    image_inputs = processor(
        images=images,
        padding=True,
        truncation=True,
        max_length=77,
        return_tensors="pt",
    ).to(device)

    text_inputs = processor(
        text=prompt,
        padding=True,
        truncation=True,
        max_length=77,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        image_embs = model.get_image_features(**image_inputs)
        image_embs = image_embs / torch.norm(image_embs, dim=-1, keepdim=True)

        text_embs = model.get_text_features(**text_inputs)
        text_embs = text_embs / torch.norm(text_embs, dim=-1, keepdim=True)

        scores = model.logit_scale.exp() * (text_embs @ image_embs.T)[0]
        probs = torch.softmax(scores, dim=-1)

    return probs.cpu().tolist()


In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Load the CSV file
data = pd.read_csv('/content/drive/MyDrive/Colab/Data/data.csv')

# Initialize an empty list to store results
results = []

# Iterate over each row in the dataframe
for index, row in data.iterrows():
    image_name = row['name']
    prompt = row['prompt']

    # Construct the path to the image file
    image_path = os.path.join('/content/drive/MyDrive/Colab/Images/AGIQA-3K', image_name)

    # Load the image
    try:
        with Image.open(image_path) as img:
            score = calc_probs(prompt, [img])
            results.append({
                'image_name': image_name,
                'prompt': prompt,
                'alignment_score': score
            })
    except IOError:
        print(f"Failed to load image {image_name}")

# Convert results to a DataFrame for easy viewing or further processing
results_df = pd.DataFrame(results)
print(results_df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Failed to load image sd1.5_lowstep_161.jpg
Failed to load image sd1.5_lowstep_162.jpg
Failed to load image sd1.5_lowstep_163.jpg
Failed to load image sd1.5_lowstep_164.jpg
Failed to load image sd1.5_lowstep_165.jpg
Failed to load image sd1.5_lowstep_166.jpg
Failed to load image sd1.5_lowstep_167.jpg
Failed to load image sd1.5_lowstep_168.jpg
Failed to load image sd1.5_lowstep_169.jpg
Failed to load image sd1.5_lowstep_170.jpg
Failed to load image sd1.5_lowstep_171.jpg
Failed to load image sd1.5_lowstep_172.jpg
Failed to load image sd1.5_lowstep_173.jpg
Failed to load image sd1.5_lowstep_174.jpg
Failed to load image sd1.5_lowstep_175.jpg
Failed to load image sd1.5_lowstep_176.jpg
Failed to load image sd1.5_lowstep_177.jpg
Failed to load image sd1.5_lowstep_178.jpg
Failed to load image sd1.5_lowstep_179.jpg
Failed to load image sd1.5_lowstep_180.jpg
Failed to l